In [1]:
import pandas as pd
# Conexion a gdrive



df_books_rating = pd.read_csv('/kaggle/input/amazon-books-reviews/Books_rating.csv')

df_book_data = pd.read_csv('/kaggle/input/amazon-books-reviews/books_data.csv')

print(df_book_data.shape)
print(df_books_rating.shape)

(212404, 10)
(3000000, 10)


In [2]:
#1 agrupar por Title, y sacar la mediana de los scores.
#hacer merge, para llevar el valor, y 

In [3]:
df_books_rating = df_books_rating.groupby('Title')['review/score'].median()
df_books_rating

Title
" Film technique, " and, " Film acting "                                                                                                   4.5
" We'll Always Have Paris": The Definitive Guide to Great Lines from the Movies                                                            5.0
"... And Poetry is Born ..." Russian Classical Poetry                                                                                      4.0
"A Titanic hero" Thomas Andrews, shipbuilder                                                                                               5.0
"A Truthful Impression of the Country": British and American Travel Writing in China, 1880-1949                                            4.0
                                                                                                                                          ... 
with an everlasting love                                                                                                                

In [4]:
# Fusionar los dos dataframes en la columna 'Title'
merged_df = pd.merge(df_book_data, df_books_rating, on='Title', how='inner')
merged_df = merged_df[['Title', 'description','authors','categories', 'publishedDate', 'review/score']]
merged_df = merged_df.dropna()

merged_df.shape

(135823, 6)

In [5]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 33.7 gigabytes of available RAM

You are using a high-RAM runtime!


In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Oct 26 04:16:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [7]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# Ensure you have GPU available
if not torch.cuda.is_available():
    raise ValueError("No GPU found, please ensure you're running on a GPU instance.")

# Use CUDA device
device = torch.device("cuda")

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

# If you have multiple GPUs, wrap the model with DataParallel
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = torch.nn.DataParallel(model)

def generate_embeddings(text):
    # Tokenize on CPU
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    # Move tokens to GPU
    tokens = {key: val.to(device) for key, val in tokens.items()}
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy().reshape(-1)

# Assuming merged_df is loaded or defined elsewhere

# Combine 'description' and 'categories' text, then generate embeddings
merged_df['combined_text'] = merged_df['description'] + " " + merged_df['categories']
merged_df['combined_embedding'] = merged_df['combined_text'].apply(generate_embeddings)

# Train the k-NN model
knn_model = NearestNeighbors(n_neighbors=6)  # Fetching 6 neighbors including the book itself
knn_model.fit(np.stack(merged_df['combined_embedding'].values))

def recommend_books(new_text):
    new_embedding = generate_embeddings(new_text)
    distances, indices = knn_model.kneighbors([new_embedding])
    recommended_books = merged_df.iloc[indices[0][1:]]  # Exclude the first index as it will be the book itself
    return recommended_books[['Title', 'authors', 'publishedDate']]

# Example usage
recommend_books('I want to read something about fantasy and love')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Using 2 GPUs!


,Title,authors,publishedDate
128893,My Own Big Bed,['Lawrence E. Shapiro'],2008
125509,Missing Beauty,['Zhi Zihuakai'],2020-04-20
59705,Australia: Search for Tomorrow/Search for Yest...,"['Eileen M. Berger', 'Colleen Coble', 'Denise ...",2000
54094,Bob Books Plus,['Bobby Lynn Maslen'],1996-08-01
86129,Poetry for the Common Man,['Steven Edward Thacker'],2019-11-12


In [8]:
#have into account the rank

def recommend_books_1(new_text):
    new_embedding = generate_embeddings(new_text)
    distances, indices = knn_model.kneighbors([new_embedding])
    
    # Normalize the review scores to [0, 1]
    max_score = merged_df['review/score'].max()
    min_score = merged_df['review/score'].min()
    merged_df['normalized_score'] = (merged_df['review/score'] - min_score) / (max_score - min_score)
    
    # Adjust distances based on review scores
    for i, index in enumerate(indices[0]):
        distances[0][i] *= (1 - merged_df.iloc[index]['normalized_score'])

    # Now, sort the books based on adjusted distances
    sorted_indices = np.argsort(distances[0])

    # Get top 5 books based on adjusted distances
    recommended_books = merged_df.iloc[sorted_indices[1:6]]  # Exclude the first index as it will be the book itself

    return recommended_books[['Title', 'authors', 'publishedDate']]


recommend_books_1('I want to read something fantasy and love')


,Title,authors,publishedDate
5,The Church of Christ: A Biblical Ecclesiology ...,['Everett Ferguson'],1996
8,Saint Hyacinth of Poland,['Mary Fabyan Windeatt'],2009-01-01
9,Rising Sons and Daughters: Life Among Japan's ...,['Steven Wardell'],1995
2,Wonderful Worship in Smaller Churches,['David R. Ray'],2000
3,Whispers of the Wicked Saints,['Veronica Haddon'],2005-02


In [9]:
#ponderar basado en el ranking.

from sklearn.metrics.pairwise import cosine_similarity

def recommend_based_on_text(new_text):
    # 1. Generar embeddings del nuevo texto
    new_embedding = generate_embeddings(new_text)

    # 2. Calcular la similitud entre el nuevo embedding y todos los embeddings de libros
    similarities = cosine_similarity([new_embedding], merged_df['combined_embedding'].tolist())

    # 3. Ponderar por rating
    # Suponiendo que 'review/score' está en el rango [0, 1]. Si no es así, normalízalo.
    weighted_scores = similarities * merged_df['review/score'].values

    # 4. Recomendar
    recommended_indices = weighted_scores.argsort()[0][::-1][:5]  # Top 5 recomendaciones
    recommended_books = merged_df.iloc[recommended_indices]
    
    return recommended_books[['Title', 'authors', 'publishedDate']]

# Uso del método
recommend_based_on_text("I want to read something fantasy and love")



,Title,authors,publishedDate
173491,Curious George's Dream (Turtleback School & Li...,"['H. A. Rey', 'Erica Zappy']",2015-06-15
128199,The Canary Chronicles: The Road to Maura,['Anne Smith Burton'],2004-10
103584,Chamario: Libro De Rimas Para Ninos (Rimas Y A...,['Parragon'],2015-04-01
55116,Rimas/ Rhymes (Clasicos / Classics) (Spanish E...,['Parragon'],2015-04-01
153173,Stuart Little (E.B. White) (Novel units),"['Novel Units', 'Anne Troy']",2000


In [10]:
import joblib

# Save BERT model
torch.save(model.state_dict(), 'bert_model.pth')

# Save k-NN model
joblib.dump(knn_model, 'knn_model.pkl')

# Save the dataframe with embeddings
merged_df.to_parquet('merged_df_embeddings.parquet') 
#merged_df.to_pickle('merged_df_embeddings.pkl')


# INFERENCE

In [ ]:
recommend_books('I want to read something fantasy and love')

In [ ]:
recommend_books('I like books about dramatic sex and Give me some romantic and war book')

In [ ]:
recommend_books('Give me some romantic and war book')

In [ ]:
recommend_books('The best books are the dark novels')

In [ ]:
merged_df.to_csv('df_embedding.csv', index=False)